In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from IPython.display import display, HTML
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm as tqdm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
df_policy = pd.read_csv("policy_claim/policy_0702.csv")
df_train = pd.read_csv("training-set.csv")
df_test = pd.read_csv("testing-set.csv")
df_test['Next_Premium']=0

In [3]:
df_policy['Cancellation_1'] = df_policy['Cancellation'].apply({'Y':1, ' ':0}.get)
df_policy['Main_Insurance_Coverage_Group_1'] = df_policy['Main_Insurance_Coverage_Group'].apply(
{'車責':0, '竊盜':1, '車損':2}.get)

In [4]:
df_policy['ibirth_1']=df_policy['ibirth'].str.extract('(19..)', expand=True)

In [5]:
df_policy['ibirth_1'].fillna(value=1968, inplace=True)
df_policy['ibirth_1'] = df_policy['ibirth_1'].apply(lambda x: 2017-int(x))

In [4]:
policy_number_mapping = {}
for idx, data in tqdm(df_policy.iterrows(), total=df_policy.shape[0]):
    if data['Policy_Number'] not in policy_number_mapping:
        policy_number_mapping[data['Policy_Number']] = []
    policy_number_mapping[data['Policy_Number']].append(idx)

In [5]:
df_policy_1 = pd.DataFrame()
temp_df_policy_1_list = []

In [6]:
def produce_stat_data(data, df, name):
    data['total_'+name] = df[name].sum()
    data['avg_'+name] = df[name].mean()
    data['max_'+name] = df[name].max()
    data['min_'+name] = df[name].min()
    data['std_'+name] = df[name].std()
    return data

In [ ]:
unique_policy_numbers = df_policy['Policy_Number'].unique().tolist()
for idx, policy_number in tqdm(enumerate(unique_policy_numbers), total=len(unique_policy_numbers)):
    idx_mapping = policy_number_mapping[policy_number]
    temp_df_policy = df_policy.iloc[idx_mapping].copy()
    temp_df_policy = temp_df_policy.sort_values('Premium')
    
    data = temp_df_policy.iloc[0].copy()
    
    data = produce_stat_data(data, temp_df_policy, 'Premium')
    
    data = produce_stat_data(data, temp_df_policy, 'Insured_Amount1')
    data = produce_stat_data(data, temp_df_policy, 'Insured_Amount2')
    data = produce_stat_data(data, temp_df_policy, 'Insured_Amount3')

    temp_df_policy['all_insured_amount'] = temp_df_policy['Insured_Amount1']+temp_df_policy['Insured_Amount2']+temp_df_policy['Insured_Amount3']
    
    data = produce_stat_data(data, temp_df_policy, 'all_insured_amount')
    
    data = produce_stat_data(data, temp_df_policy, 'Coverage_Deductible_if_applied')
    
    data['count_insurance_coverage'] = temp_df_policy.shape[0]
    
    data['count_cancellation_1'] = temp_df_policy['Cancellation_1'].sum()
    
    data['previous_total_premium'] = 0
    if temp_df_policy[:1]['Prior_Policy_Number'].isnull().iloc[0]==False:
        if temp_df_policy.iloc[0]['Prior_Policy_Number'] in policy_number_mapping:
            idx_mapping_previous = policy_number_mapping[temp_df_policy.iloc[0]['Prior_Policy_Number']]
            temp_previous_df_policy = df_policy.iloc[idx_mapping_previous].copy()
            data['previous_total_premium'] = temp_previous_df_policy['Premium'].sum()
    
    temp_df_policy_1_list.append(data)
    
#     if idx==10:
#         break

In [ ]:
import pickle
file = open('temp_df_policy_1_list.pickle', 'wb')
pickle.dump(temp_df_policy_1_list, file)
file.close()

In [ ]:
df_policy_1 = pd.DataFrame.from_dict(temp_df_policy_1_list)

In [ ]:
df_policy_1.head()

In [ ]:
df_policy_1['ratio_cancellation_1'] = df_policy_1['count_cancellation_1'] / df_policy_1['count_insurance_coverage']

In [ ]:
df_policy_1['dbirth_1'] = df_policy_1['dbirth'].str.split('/').str.get(1)
df_policy_1['dbirth_1'].fillna(value=1968, inplace=True)
df_policy_1['dbirth_1'] = df_policy_1['dbirth_1'].astype(int)

In [ ]:
del policy_number_mapping
del df_policy

In [8]:
df_claim = pd.read_csv("policy_claim/claim_0702.csv")

In [9]:
claim_number_mapping = {}
for idx, data in tqdm(df_claim.iterrows(), total=df_claim.shape[0]):
    if data['Policy_Number'] not in claim_number_mapping:
        claim_number_mapping[data['Policy_Number']] = []
    claim_number_mapping[data['Policy_Number']].append(idx)

In [ ]:
# first add columns
for idx, data in tqdm(df_policy_1.iterrows(), total=df_policy_1.shape[0]):
    policy_number = data['Policy_Number']
    if policy_number in claim_number_mapping:
        idx_mapping = claim_number_mapping[policy_number]
        temp_df_claim = df_claim.iloc[idx_mapping].copy()

        df_policy_1.loc[idx, 'count_Claim'] = temp_df_claim.shape[0]
        
        df_policy_1.loc[idx, 'ratio_Nature_of_the_claim'] = np.sum(temp_df_claim['Nature_of_the_claim']==1)/temp_df_claim.shape[0]
        
        data = {}
        
        data = produce_stat_data(data, temp_df_claim, 'At_Fault?')
        
        data = produce_stat_data(data, temp_df_claim, 'Paid_Loss_Amount')
        
        data = produce_stat_data(data, temp_df_claim, 'paid_Expenses_Amount')
        
        data = produce_stat_data(data, temp_df_claim, 'Salvage_or_Subrogation?')
        
        data = produce_stat_data(data, temp_df_claim, 'Deductible')
        
        for column_name in list(data.keys()): 
            df_policy_1[column_name] = None
        df_policy_1.loc[idx, list(data.keys())] = list(data.values())
        print(list(data.keys()))
        print(list(data.values()))
        break

In [ ]:
for idx, data in tqdm(df_policy_1.iterrows(), total=df_policy_1.shape[0]):
    policy_number = data['Policy_Number']
    if policy_number in claim_number_mapping:
        idx_mapping = claim_number_mapping[policy_number]
        temp_df_claim = df_claim.iloc[idx_mapping].copy()

        df_policy_1.loc[idx, 'count_Claim'] = temp_df_claim.shape[0]
        
        df_policy_1.loc[idx, 'ratio_Nature_of_the_claim'] = np.sum(temp_df_claim['Nature_of_the_claim']==1)/temp_df_claim.shape[0]
        
        data = {}
        
        data = produce_stat_data(data, temp_df_claim, 'At_Fault?')
        
        data = produce_stat_data(data, temp_df_claim, 'Paid_Loss_Amount')
        
        data = produce_stat_data(data, temp_df_claim, 'paid_Expenses_Amount')
        
        data = produce_stat_data(data, temp_df_claim, 'Salvage_or_Subrogation?')
        
        data = produce_stat_data(data, temp_df_claim, 'Deductible')

        df_policy_1.loc[idx, list(data.keys())] = list(data.values())

In [ ]:
df_policy_1.to_csv('policy_df_0802.csv', index=False)

In [ ]:
df_policy_1 = pd.read_csv('output/policy_df_0804.csv')

In [11]:
unique_policy_numbers = df_policy['Policy_Number'].unique().tolist()
unique_insurance_coverages = df_policy['Insurance_Coverage'].unique().tolist()

insurance_coverages_df = pd.DataFrame()
temp_insurance_coverages_df_list = []
for insurance_type in unique_insurance_coverages:
    insurance_coverages_df['Policy_Number'] = None
    insurance_coverages_df[insurance_type] = None
    insurance_coverages_df[insurance_type+'_a1'] = None
    insurance_coverages_df[insurance_type+'_a2'] = None
    insurance_coverages_df[insurance_type+'_a3'] = None
    insurance_coverages_df[insurance_type+'_all'] = None
    insurance_coverages_df[insurance_type+'_p'] = None
    insurance_coverages_df[insurance_type+'_ap_ratio'] = None

In [12]:
for idx, policy_number in tqdm(enumerate(unique_policy_numbers), total=len(unique_policy_numbers)):
    idx_mapping = policy_number_mapping[policy_number]
    temp_df_policy = df_policy.iloc[idx_mapping].copy()
    
    append_dict = {}
    append_dict['Policy_Number'] = policy_number
    for idx2, data in temp_df_policy.iterrows():
        insurance_type = data['Insurance_Coverage']
        append_dict[insurance_type] = 1
        append_dict[insurance_type+'_a1'] = data['Insured_Amount1']
        append_dict[insurance_type+'_a2'] = data['Insured_Amount2']
        append_dict[insurance_type+'_a3'] = data['Insured_Amount3']
        append_dict[insurance_type+'_all'] = data['Insured_Amount1']+data['Insured_Amount2']+data['Insured_Amount3']
        append_dict[insurance_type+'_p'] = data['Premium']
        append_dict[insurance_type+'_ap_ratio'] = append_dict[insurance_type+'_all']/(append_dict[insurance_type+'_p']+0.000000001)
    temp_insurance_coverages_df_list.append(append_dict)
    
#     if idx > 100:
#         break

In [13]:
import pickle
file = open('temp_insurance_coverages_df_list.pickle', 'wb')
pickle.dump(temp_insurance_coverages_df_list, file)
file.close()

In [14]:
insurance_coverages_df = pd.DataFrame.from_dict(temp_insurance_coverages_df_list)
insurance_coverages_df.head()

,00I,00I_a1,00I_a2,00I_a3,00I_all,00I_ap_ratio,00I_p,01A,01A_a1,01A_a2,01A_a3,01A_all,01A_ap_ratio,01A_p,01J,01J_a1,01J_a2,01J_a3,01J_all,01J_ap_ratio,01J_p,02K,02K_a1,02K_a2,02K_a3,02K_all,02K_ap_ratio,02K_p,03L,03L_a1,03L_a2,03L_a3,03L_all,03L_ap_ratio,03L_p,04M,04M_a1,04M_a2,04M_a3,04M_all,04M_ap_ratio,04M_p,05E,05E_a1,05E_a2,05E_a3,05E_all,05E_ap_ratio,05E_p,05N,05N_a1,05N_a2,05N_a3,05N_all,05N_ap_ratio,05N_p,06F,06F_a1,06F_a2,06F_a3,06F_all,06F_ap_ratio,06F_p,07P,07P_a1,07P_a2,07P_a3,07P_all,07P_ap_ratio,07P_p,08H,08H_a1,08H_a2,08H_a3,08H_all,08H_ap_ratio,08H_p,09@,09@_a1,09@_a2,09@_a3,09@_all,09@_ap_ratio,09@_p,09I,09I_a1,09I_a2,09I_a3,09I_all,09I_ap_ratio,09I_p,10A,10A_a1,10A_a2,10A_a3,10A_all,10A_ap_ratio,10A_p,12L,12L_a1,12L_a2,12L_a3,12L_all,12L_ap_ratio,12L_p,14E,14E_a1,14E_a2,14E_a3,14E_all,14E_ap_ratio,14E_p,14N,14N_a1,14N_a2,14N_a3,14N_all,14N_ap_ratio,14N_p,15F,15F_a1,15F_a2,15F_a3,15F_all,15F_ap_ratio,15F_p,15O,15O_a1,15O_a2,15O_a3,15O_all,15O_ap_ratio,15O_p,16G,16G_a1,16G_a2,16G_a3,16G_all,16G_ap_ratio,16G_p,16P,16P_a1,16P_a2,16P_a3,16P_all,16P_ap_ratio,16P_p,18@,18@_a1,18@_a2,18@_a3,18@_all,18@_ap_ratio,18@_p,18I,18I_a1,18I_a2,18I_a3,18I_all,18I_ap_ratio,18I_p,20B,20B_a1,20B_a2,20B_a3,20B_all,20B_ap_ratio,20B_p,20K,20K_a1,20K_a2,20K_a3,20K_all,20K_ap_ratio,20K_p,25G,25G_a1,25G_a2,25G_a3,25G_all,25G_ap_ratio,25G_p,26H,26H_a1,26H_a2,26H_a3,26H_all,26H_ap_ratio,26H_p,27I,27I_a1,27I_a2,27I_a3,27I_all,27I_ap_ratio,27I_p,29B,29B_a1,29B_a2,29B_a3,29B_all,29B_ap_ratio,29B_p,29K,29K_a1,29K_a2,29K_a3,29K_all,29K_ap_ratio,29K_p,32N,32N_a1,32N_a2,32N_a3,32N_all,32N_ap_ratio,32N_p,33F,33F_a1,33F_a2,33F_a3,33F_all,33F_ap_ratio,33F_p,33O,33O_a1,33O_a2,33O_a3,33O_all,33O_ap_ratio,33O_p,34P,34P_a1,34P_a2,34P_a3,34P_all,34P_ap_ratio,34P_p,35H,35H_a1,35H_a2,35H_a3,35H_all,35H_ap_ratio,35H_p,36I,36I_a1,36I_a2,36I_a3,36I_all,36I_ap_ratio,36I_p,37J,37J_a1,37J_a2,37J_a3,37J_all,37J_ap_ratio,37J_p,40M,40M_a1,40M_a2,40M_a3,40M_all,40M_ap_ratio,40M_p,41E,41E_a1,41E_a2,41E_a3,41E_all,41E_ap_ratio,41E_p,41N,41N_a1,41N_a2,41N_a3,41N_all,41N_ap_ratio,41N_p,42F,42F_a1,42F_a2,42F_a3,42F_all,42F_ap_ratio,42F_p,45@,45@_a1,45@_a2,45@_a3,45@_all,45@_ap_ratio,45@_p,46A,46A_a1,46A_a2,46A_a3,46A_all,46A_ap_ratio,46A_p,47B,47B_a1,47B_a2,47B_a3,47B_all,47B_ap_ratio,47B_p,51O,51O_a1,51O_a2,51O_a3,51O_all,51O_ap_ratio,51O_p,55J,55J_a1,55J_a2,55J_a3,55J_all,55J_ap_ratio,55J_p,56B,56B_a1,56B_a2,56B_a3,56B_all,56B_ap_ratio,56B_p,56K,56K_a1,56K_a2,56K_a3,56K_all,56K_ap_ratio,56K_p,57C,57C_a1,57C_a2,57C_a3,57C_all,57C_ap_ratio,57C_p,57L,57L_a1,57L_a2,57L_a3,57L_all,57L_ap_ratio,57L_p,65K,65K_a1,65K_a2,65K_a3,65K_all,65K_ap_ratio,65K_p,66C,66C_a1,66C_a2,66C_a3,66C_all,66C_ap_ratio,66C_p,66L,66L_a1,66L_a2,66L_a3,66L_all,66L_ap_ratio,66L_p,67D,67D_a1,67D_a2,67D_a3,67D_all,67D_ap_ratio,67D_p,68E,68E_a1,68E_a2,68E_a3,68E_all,68E_ap_ratio,68E_p,68N,68N_a1,68N_a2,68N_a3,68N_all,68N_ap_ratio,68N_p,70G,70G_a1,70G_a2,70G_a3,70G_all,70G_ap_ratio,70G_p,70P,70P_a1,70P_a2,70P_a3,70P_all,70P_ap_ratio,70P_p,71H,71H_a1,71H_a2,71H_a3,71H_all,71H_ap_ratio,71H_p,72@,72@_a1,72@_a2,72@_a3,72@_all,72@_ap_ratio,72@_p,Policy_Number
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2000000.0,0.0,10000000.0,12000000.0,7736.943907,1551.0,1.0,0.0,0.0,300000.0,300000.0,257.953568,1163.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [15]:
insurance_coverages_df.to_csv('insurance_coverages_df.csv', index=False)

In [16]:
insurance_coverages_df.head()

,00I,00I_a1,00I_a2,00I_a3,00I_all,00I_ap_ratio,00I_p,01A,01A_a1,01A_a2,01A_a3,01A_all,01A_ap_ratio,01A_p,01J,01J_a1,01J_a2,01J_a3,01J_all,01J_ap_ratio,01J_p,02K,02K_a1,02K_a2,02K_a3,02K_all,02K_ap_ratio,02K_p,03L,03L_a1,03L_a2,03L_a3,03L_all,03L_ap_ratio,03L_p,04M,04M_a1,04M_a2,04M_a3,04M_all,04M_ap_ratio,04M_p,05E,05E_a1,05E_a2,05E_a3,05E_all,05E_ap_ratio,05E_p,05N,05N_a1,05N_a2,05N_a3,05N_all,05N_ap_ratio,05N_p,06F,06F_a1,06F_a2,06F_a3,06F_all,06F_ap_ratio,06F_p,07P,07P_a1,07P_a2,07P_a3,07P_all,07P_ap_ratio,07P_p,08H,08H_a1,08H_a2,08H_a3,08H_all,08H_ap_ratio,08H_p,09@,09@_a1,09@_a2,09@_a3,09@_all,09@_ap_ratio,09@_p,09I,09I_a1,09I_a2,09I_a3,09I_all,09I_ap_ratio,09I_p,10A,10A_a1,10A_a2,10A_a3,10A_all,10A_ap_ratio,10A_p,12L,12L_a1,12L_a2,12L_a3,12L_all,12L_ap_ratio,12L_p,14E,14E_a1,14E_a2,14E_a3,14E_all,14E_ap_ratio,14E_p,14N,14N_a1,14N_a2,14N_a3,14N_all,14N_ap_ratio,14N_p,15F,15F_a1,15F_a2,15F_a3,15F_all,15F_ap_ratio,15F_p,15O,15O_a1,15O_a2,15O_a3,15O_all,15O_ap_ratio,15O_p,16G,16G_a1,16G_a2,16G_a3,16G_all,16G_ap_ratio,16G_p,16P,16P_a1,16P_a2,16P_a3,16P_all,16P_ap_ratio,16P_p,18@,18@_a1,18@_a2,18@_a3,18@_all,18@_ap_ratio,18@_p,18I,18I_a1,18I_a2,18I_a3,18I_all,18I_ap_ratio,18I_p,20B,20B_a1,20B_a2,20B_a3,20B_all,20B_ap_ratio,20B_p,20K,20K_a1,20K_a2,20K_a3,20K_all,20K_ap_ratio,20K_p,25G,25G_a1,25G_a2,25G_a3,25G_all,25G_ap_ratio,25G_p,26H,26H_a1,26H_a2,26H_a3,26H_all,26H_ap_ratio,26H_p,27I,27I_a1,27I_a2,27I_a3,27I_all,27I_ap_ratio,27I_p,29B,29B_a1,29B_a2,29B_a3,29B_all,29B_ap_ratio,29B_p,29K,29K_a1,29K_a2,29K_a3,29K_all,29K_ap_ratio,29K_p,32N,32N_a1,32N_a2,32N_a3,32N_all,32N_ap_ratio,32N_p,33F,33F_a1,33F_a2,33F_a3,33F_all,33F_ap_ratio,33F_p,33O,33O_a1,33O_a2,33O_a3,33O_all,33O_ap_ratio,33O_p,34P,34P_a1,34P_a2,34P_a3,34P_all,34P_ap_ratio,34P_p,35H,35H_a1,35H_a2,35H_a3,35H_all,35H_ap_ratio,35H_p,36I,36I_a1,36I_a2,36I_a3,36I_all,36I_ap_ratio,36I_p,37J,37J_a1,37J_a2,37J_a3,37J_all,37J_ap_ratio,37J_p,40M,40M_a1,40M_a2,40M_a3,40M_all,40M_ap_ratio,40M_p,41E,41E_a1,41E_a2,41E_a3,41E_all,41E_ap_ratio,41E_p,41N,41N_a1,41N_a2,41N_a3,41N_all,41N_ap_ratio,41N_p,42F,42F_a1,42F_a2,42F_a3,42F_all,42F_ap_ratio,42F_p,45@,45@_a1,45@_a2,45@_a3,45@_all,45@_ap_ratio,45@_p,46A,46A_a1,46A_a2,46A_a3,46A_all,46A_ap_ratio,46A_p,47B,47B_a1,47B_a2,47B_a3,47B_all,47B_ap_ratio,47B_p,51O,51O_a1,51O_a2,51O_a3,51O_all,51O_ap_ratio,51O_p,55J,55J_a1,55J_a2,55J_a3,55J_all,55J_ap_ratio,55J_p,56B,56B_a1,56B_a2,56B_a3,56B_all,56B_ap_ratio,56B_p,56K,56K_a1,56K_a2,56K_a3,56K_all,56K_ap_ratio,56K_p,57C,57C_a1,57C_a2,57C_a3,57C_all,57C_ap_ratio,57C_p,57L,57L_a1,57L_a2,57L_a3,57L_all,57L_ap_ratio,57L_p,65K,65K_a1,65K_a2,65K_a3,65K_all,65K_ap_ratio,65K_p,66C,66C_a1,66C_a2,66C_a3,66C_all,66C_ap_ratio,66C_p,66L,66L_a1,66L_a2,66L_a3,66L_all,66L_ap_ratio,66L_p,67D,67D_a1,67D_a2,67D_a3,67D_all,67D_ap_ratio,67D_p,68E,68E_a1,68E_a2,68E_a3,68E_all,68E_ap_ratio,68E_p,68N,68N_a1,68N_a2,68N_a3,68N_all,68N_ap_ratio,68N_p,70G,70G_a1,70G_a2,70G_a3,70G_all,70G_ap_ratio,70G_p,70P,70P_a1,70P_a2,70P_a3,70P_all,70P_ap_ratio,70P_p,71H,71H_a1,71H_a2,71H_a3,71H_all,71H_ap_ratio,71H_p,72@,72@_a1,72@_a2,72@_a3,72@_all,72@_ap_ratio,72@_p,Policy_Number
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2000000.0,0.0,10000000.0,12000000.0,7736.943907,1551.0,1.0,0.0,0.0,300000.0,300000.0,257.953568,1163.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,